### Phần 1:


### Phần 2:
##### Chuyển dữ liệu host ↔ device:
Host đưa input cho device. Device xử lí trong nội bộ và trả về output cho host. Vậy ta sẽ có các hàm cudaMemcpy để thực hiện các tác vụ:  
+ chuyển input từ host sang device  
+ Khởi tạo các trọng số cho mô hình sẽ được thực hiện trên host rồi chuyển sang device.
+ Chuyển output từ device sang host
+ Chuyển giá trị hàm loss từ device sang host để giám sát quá trình huấn luyện mô hình.

##### Phần tổ chức dữ liệu cho kernel:
Như ta đã biết thì dữ liệu lưu theo index của `threadIdx`, `blockDim`, `blockIdx` chỉ có 3 chiều (x, y, z), trong khi đó, dữ liệu mà ta đang xử lí có tới 4 chiều (H, W, Channel, Batch). Ý tưởng của nhóm em là gộp chiều Channel với Batch lại và lưu và chiều z. Khi đó, một dữ liệu input có chiều (x,y,c,b):
$$\begin{align}
\text{flat\_index} &= (b \cdot C + c) \cdot H \cdot W + W \cdot h + w \\
&= ((b \cdot C + c) \cdot H + h) \cdot W + w
\end{align}$$

##### Tại sao trên các kernel ReLU lại chỉ sử dụng 1 chiều x trong khi các kernel khác thì sử dụng tới 3 chiều ?

##### Tại sao index của chiều batch-channel lại được biểu diễn qua blockIdx.z mà không phải là blockIdx.z * blockDim.z + threadIdx.z ?


##### Tại sao ko cần cudaDeviceSynchronize() sau mỗi kernel call trong forwardPass ?
Vì tất cả job đều nằm trên cùng 1 stream 0 nên các hàm được nằm ở trước chắc chắn sẽ được thực hiện trước.


##### Thắc mắc về vấn đề đo thời gian kernel:
1. Tại sao ta phải sử dụng lambda function cho đoạn code ?
```c++
measureKernelTime([&]() {
        k_conv2d_forward<<<grid, block>>>(d_input, d_w_enc_conv1, d_b_enc_conv1, d_enc_conv1, inC, outC, H_in, W_in, filterWidth, padding, stride);
    }, conv_forward_time, "Conv2D Forward");
```
Vì:
Immediate execution: When you write the kernel launch directly as a parameter, it gets executed immediately when the function arguments are evaluated, before measureKernelTime even starts running


##### Tại sao backward lại lâu nhất ?



##### Khó khăn:
Trong quá trình train, liên tục bị gradient explosion nên đã áp dụng gradient clipping cho backprop ổn định hơn

#### Cải tiến:

##### Apply shared memory cho kernel backprop cho các weight:
Ý tưởng: Mỗi thread xử lí một backprop cho một output. Từ một output xác định (ob, oc, oh, ow), ta thực hiện backprop cho từng filter thuộc từng channel bằng cách 
nhân grad với các input tương ứng.  
- First approach: Sử dụng memory để load toàn bộ tensor 3 chiều của input ứng với output mà thread đang xử lí.
-> Bị tràn shared mem khi blockSize = 32. fail silently -> tốn rất nhiều thời gian mới phát hiện ra
- Second approach: Chia ra xử lí từng channel input
-> good

##### Apply shared memory cho kernel backprop input:
Ý tưởng: Mỗi threadd xử lí backprop cho một input. Từ một input xác định (ib, ic, ih, iw), ta xác định ra các output mà input đó có đóng góp và thực hiện backprop 
bằng cách nhân grad với các weight tương ứng